In [2]:
import pandas as pd
from datasets import Dataset

dataset_name = "irony"

assert dataset_name in ["irony", "misogyny", "sentiment"]

if dataset_name == "irony":
    fname = "./irony.csv"
    dataset = pd.read_csv(fname, sep=";")
    
    dataset["target"] = ""
    dataset.loc[dataset["irony"] == 1, "target"] = "si"
    dataset.loc[dataset["irony"] == 0, "target"] = "no"

    dataset["text"] = dataset["text"]

    classes = ["si", "no"]

elif dataset_name == "misogyny":
    fname = "./misogyny.tsv"
    class_col = "misogynous"
    dataset = pd.read_csv(fname, sep="\t")
    
    dataset["target"] = ""
    dataset.loc[dataset["misogynous"] == 1, "target"] = "si"
    dataset.loc[dataset["misogynous"] == 0, "target"] = "no"
    
    dataset["text"] = dataset["text"]
    
    classes = ["si", "no"]

elif dataset_name == "sentiment":
    fname = "./sentiment.csv"
    class_col = "sentiment"
    dataset = pd.read_csv(fname, sep=";")
    
    dataset["target"] = "neutrale"
    dataset.loc[dataset["positivo"] == 1, "target"] = "positivo"
    dataset.loc[dataset["negativo"] == 1, "target"] = "negativo"

    dataset["text"] = dataset["tweet"]

    classes = ["positivo", "negativo", "neutrale"]


dataset = Dataset.from_pandas(dataset)
dataset = dataset.train_test_split(test_size=0.8, seed=42)

In [3]:
from get_model import get_model
model_name = "saiga-7b"

DEVICE = "cuda:0"

model, tokenizer = get_model(model_name)
model = model.to(DEVICE)

Loading checkpoint shards: 100%|██████████| 2/2 [02:21<00:00, 70.82s/it]


In [4]:
from get_prompt import get_prompt

generate_prompt, stop = get_prompt(model_name)

In [5]:
def build_question(context):
    if dataset_name == "irony":
        return f"Dato questo testo:\n{context}\nIl suo contenuto è ironico? (si/no)"
    elif dataset_name == "misogyny":
        return f"Dato questo testo:\n{context}\nIl suo contenuto è misogino? (si/no)"
    elif dataset_name == "sentiment":
        return f"Dato questo testo:\n{context}\nIl suo contenuto è positivo, negativo o neutrale?"
    

In [6]:
import random
def get_shots(dataset, n):
    conversation = []
    for i in range(n):
        elem = random.choice(dataset["train"])
        q_shot = build_question(elem["text"])
        conversation.append(dict(
            role="user",
            text=q_shot
        ))
        conversation.append(dict(
            role="ai",
            text=elem["target"]
        ))
    return conversation

shots = get_shots(dataset, 3)
print(shots)

[{'role': 'user', 'text': 'Dato questo testo:\n#labuonascuola di #Renzi dal pubblico al privato con 60 € almese <URL>\nIl suo contenuto è ironico? (si/no)'}, {'role': 'ai', 'text': 'si'}, {'role': 'user', 'text': "Dato questo testo:\nHo capito che l'isis vuole convertire tutti all'islam però potrebbe farlo senza uccidere e obbligazioni..\nIl suo contenuto è ironico? (si/no)"}, {'role': 'ai', 'text': 'si'}, {'role': 'user', 'text': 'Dato questo testo:\n<URL> anche a lui nn piace #labuonascuola #MIDAperRUOLO\nIl suo contenuto è ironico? (si/no)'}, {'role': 'ai', 'text': 'no'}]


In [7]:
import torch

def forward_model(*, prompt, model, tokenizer):
    if "pad_token" not in tokenizer.special_tokens_map:
        tokenizer.pad_token = tokenizer.eos_token

    input_ids = tokenizer(
        prompt,
        return_tensors='pt',
        padding=True,
    ).input_ids
    with torch.no_grad():
        input_ids = input_ids.to(DEVICE)
        output_ids = model.generate(
            input_ids=input_ids,
            max_new_tokens=32,
            pad_token_id=tokenizer.eos_token_id
        )

    ret = []
    for i in range(0, len(output_ids)):
        generated_text = tokenizer.decode(
            output_ids[i],
            skip_special_tokens=True
        )
        generated_text = generated_text[len(prompt[i]):]

        if "\n" in generated_text:
            generated_text = generated_text[:generated_text.index("\n")]
        
        ret.append(generated_text.strip())

    return ret

In [8]:
import uuid
from tqdm import tqdm

batch_size = 16

predicted_answers = []
theoretical_answers = []

bar = tqdm(dataset["test"].shuffle().iter(batch_size=batch_size), total=len(dataset["test"]))
total = 0
for elem in bar:
    try:
        ids = []
        texts = elem["text"]
        targets = elem["target"]

        model_inputs = []
        for i in texts:
            ids.append(str(uuid.uuid4()))
            shots = get_shots(dataset, 5)
            model_inputs.append(
                generate_prompt(
                    shots + [
                        dict(
                            role="user",
                            text=build_question(i)
                        )
                    ], do_continue=True
                )
            )

        model_outputs = forward_model(
            prompt=model_inputs,
            model=model,
            tokenizer=tokenizer,
        )

        for model_output, id in zip(model_outputs, ids):
            predicted_answers.append(dict(
                id=id,
                prediction_text=model_output,
            ))

        for o, id in zip(targets, ids):
            theoretical_answers.append(dict(
                id=id,
                answer=o,
            ))
        
    except Exception as e:
        print(e)
        continue

    bar.update(len(texts))
    total += len(texts)


import json
import os

# Create the directory if it doesn't exist
os.makedirs('./cache', exist_ok=True)

with open(f"./cache/generated-{dataset_name}-{model_name}.json", "w") as f:
    json.dump(dict(
        predicted_answers=predicted_answers,
        theoretical_answers=theoretical_answers,
    ), f, indent=4)


  6%|▋         | 44/698 [24:32:51<364:52:09, 2008.45s/it]


In [9]:
import json
with open(f"./cache/generated-{dataset_name}-{model_name}.json", "r") as f:
    data = json.load(f)

print(len(data["predicted_answers"]))

698


In [10]:
from Levenshtein import distance

y_true = []
y_pred = []

def convert_class_to_int(c):
    c = c.lower().strip()
    if c in classes:
        return classes.index(c)
    else:
        min_dist = 100000
        min_class = None
        for cl in classes:
            d = distance(c, cl)
            if d < min_dist:
                min_dist = d
                min_class = cl
        return classes.index(min_class)
        


for p, t in zip(data["predicted_answers"], data["theoretical_answers"]):
    y_true.append(convert_class_to_int(t["answer"]))
    y_pred.append(convert_class_to_int(p["prediction_text"]))

from sklearn.metrics import classification_report

print("=== REPORT ===")
print("Dataset name:", dataset_name)
print("Model name:", model_name)
print("Samples: ", len(y_true))
print()
print(classification_report(y_true, y_pred, target_names=classes))
print("==============")

=== REPORT ===
Dataset name: irony
Model name: saiga-7b
Samples:  698

              precision    recall  f1-score   support

          si       0.73      0.65      0.69       357
          no       0.67      0.75      0.71       341

    accuracy                           0.70       698
   macro avg       0.70      0.70      0.70       698
weighted avg       0.70      0.70      0.70       698



In [11]:
import os

results_dir = "./results/"

os.makedirs(results_dir, exist_ok=True)
fname = f"{results_dir}/evalita-{dataset_name}-{model_name}.json"

report = classification_report(y_true, y_pred, target_names=classes, output_dict=True)
with open(fname, "w") as f:
    json.dump(report, f, indent=4)